In [1]:
# ============================================================
# 02_select_scenarios.ipynb
# Selección de 4 escenarios diarios representativos (2024)
# Criterio: cuartiles de la desviación típica diaria (volatilidad)
# Salida: data/processed/scenarios_qstd_2024.csv
# ============================================================

# -------- 0) Montar Google Drive --------
from google.colab import drive
drive.mount('/content/drive')

# -------- 1) Imports --------
!pip -q install pandas python-dateutil

import os
import pandas as pd

# -------- 2) Rutas --------
BASE_PATH = "/content/drive/MyDrive/energy_storage_esios"
assert os.path.exists(BASE_PATH), "BASE_PATH no existe. ¿Drive montado?"

IN_PATH = os.path.join(BASE_PATH, "data", "processed", "esios_price_spot_es_2024.csv")
assert os.path.exists(IN_PATH), f"No existe el archivo de entrada: {IN_PATH}"

OUT_DIR = os.path.join(BASE_PATH, "data", "processed")
os.makedirs(OUT_DIR, exist_ok=True)

OUT_SCENARIOS = os.path.join(OUT_DIR, "scenarios_qstd_2024.csv")
OUT_DAILY_STATS = os.path.join(OUT_DIR, "daily_stats_2024.csv")

print("Entrada:", IN_PATH)
print("Salida escenarios:", OUT_SCENARIOS)

# -------- 3) Cargar serie horaria (UTC) --------
df = pd.read_csv(IN_PATH, parse_dates=["datetime"])
df = df.sort_values("datetime").reset_index(drop=True)

# Recorte estricto a 2024 (UTC) para evitar fila frontera 2025-01-01 00:00
df = df[(df["datetime"] >= "2024-01-01T00:00:00Z") & (df["datetime"] < "2025-01-01T00:00:00Z")].copy()

print("Filas tras recorte a 2024 UTC:", len(df))
print("Rango UTC:", df["datetime"].min(), "->", df["datetime"].max())

# -------- 4) Convertir a hora local Europe/Madrid --------
# Nota: la conversión genera días con 23/25 horas por cambio horario.
df["dt_local"] = df["datetime"].dt.tz_convert("Europe/Madrid")
df["date_local"] = df["dt_local"].dt.date
df["hour_local"] = df["dt_local"].dt.hour  # 0-23

# -------- 5) Estadísticos diarios (en hora local) --------
daily = (
    df.groupby("date_local")["price_eur_mwh"]
      .agg(mean="mean", std="std", min="min", max="max", count="count")
      .reset_index()
)

# Guardamos stats para justificar escenarios en memoria
daily.to_csv(OUT_DAILY_STATS, index=False)
print("Guardado daily stats:", OUT_DAILY_STATS)
print(daily.head())

# -------- 6) Control de calidad: días con 23/24/25 horas --------
counts = daily["count"].value_counts().sort_index()
print("\nDistribución de horas por día (hora local):")
print(counts)

# Para seleccionar escenarios representativos, preferimos días "normales" (24 horas)
daily_24 = daily[daily["count"] == 24].copy()
assert len(daily_24) > 300, "Hay muy pocos días con 24h; algo raro en la conversión."

# -------- 7) Cuartiles por desviación típica --------
# Q1: baja volatilidad, Q4: alta volatilidad
daily_24["std_quartile"] = pd.qcut(daily_24["std"], q=4, labels=[1,2,3,4])

# Selección representativa por cuartil:
# elegimos el día cuya std está más cerca de la mediana del cuartil (día "central")
scenarios = []
for q in [1,2,3,4]:
    subset = daily_24[daily_24["std_quartile"] == q].copy()
    target = subset["std"].median()
    subset["dist_to_median"] = (subset["std"] - target).abs()
    chosen = subset.sort_values("dist_to_median").iloc[0]
    scenarios.append(chosen)

scenarios_df = pd.DataFrame(scenarios).sort_values("std_quartile").reset_index(drop=True)

print("\nDías seleccionados (1 por cuartil de std):")
print(scenarios_df[["date_local","std_quartile","mean","std","min","max","count"]])

# -------- 8) Construir perfiles horarios (24h) de cada escenario --------
# Queremos una fila por escenario y columnas h00..h23
profiles = []
for _, row in scenarios_df.iterrows():
    d = row["date_local"]
    sub = df[df["date_local"] == d].copy()

    # Nos quedamos solo con días 24h (por seguridad)
    if len(sub) != 24:
        # Si por cualquier razón no es 24, lo saltamos (aunque no debería pasar)
        continue

    # Orden por hora local
    sub = sub.sort_values("hour_local")

    profile = {
        "date_local": str(d),
        "std_quartile": int(row["std_quartile"]),
        "mean_daily": float(row["mean"]),
        "std_daily": float(row["std"]),
    }
    # columnas horarias
    for h, v in zip(sub["hour_local"].tolist(), sub["price_eur_mwh"].tolist()):
        profile[f"h{h:02d}"] = float(v)

    profiles.append(profile)

profiles_df = pd.DataFrame(profiles).sort_values("std_quartile").reset_index(drop=True)

assert len(profiles_df) == 4, "No se han podido construir los 4 perfiles 24h."

# -------- 9) Guardar escenarios --------
profiles_df.to_csv(OUT_SCENARIOS, index=False)
print("\nGuardado escenarios:", OUT_SCENARIOS)
profiles_df


Mounted at /content/drive
Entrada: /content/drive/MyDrive/energy_storage_esios/data/processed/esios_price_spot_es_2024.csv
Salida escenarios: /content/drive/MyDrive/energy_storage_esios/data/processed/scenarios_qstd_2024.csv
Filas tras recorte a 2024 UTC: 8784
Rango UTC: 2024-01-01 00:00:00+00:00 -> 2024-12-31 23:00:00+00:00
Guardado daily stats: /content/drive/MyDrive/energy_storage_esios/data/processed/daily_stats_2024.csv
   date_local       mean        std    min     max  count
0  2024-01-01  35.064783  19.378236   1.73   60.90     23
1  2024-01-02  40.907917  16.100042   3.99   62.45     24
2  2024-01-03  55.084167  38.182071   2.21  113.26     24
3  2024-01-04  91.357917  10.918632  66.86  105.18     24
4  2024-01-05  68.615417  16.743516  30.55   90.88     24

Distribución de horas por día (hora local):
count
1       1
23      2
24    363
25      1
Name: count, dtype: int64

Días seleccionados (1 por cuartil de std):
   date_local  std_quartile        mean        std    min     

,date_local,std_quartile,mean_daily,std_daily,h00,h01,h02,h03,h04,h05,...,h14,h15,h16,h17,h18,h19,h20,h21,h22,h23
0,2024-03-07,1,8.522917,9.676325,14.13,4.89,3.51,3.20,3.20,3.20,...,0.43,0.43,0.76,3.20,5.48,24.67,35.00,30.00,16.00,5.33
1,2024-07-31,2,105.341250,17.782959,115.82,112.14,111.14,112.14,112.44,113.03,...,81.71,80.01,79.59,79.80,94.37,111.14,117.53,142.48,132.40,116.98
2,2024-04-28,3,28.722500,25.055756,56.39,38.96,39.60,35.01,35.01,35.50,...,0.00,0.00,-0.01,0.00,20.66,35.01,58.88,78.56,72.21,60.65
3,2024-10-13,4,52.750417,36.788800,69.78,62.91,57.00,58.69,55.00,60.87,...,0.00,0.00,6.80,31.28,62.82,93.08,116.97,121.28,102.78,93.56
